In [ ]:
## IMPORTS
import pandas as pd
import pandasql as ps
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil import parser
import datetime

In [ ]:
df_user_info =  pd.read_csv("user_info.csv")
df_transaction =  pd.read_csv("transaction_info.csv")


In [60]:
# a)

QUERY = '''
        WITH all_info as(
        SELECT * , df_transaction.user_id as USER_ID, SUM(amount_in_eur) as DEPOSITS 
        FROM df_user_info 
        INNER JOIN df_transaction
        ON df_user_info.user_id = df_transaction.user_id
        WHERE event_kind == "fiat_deposit"
        GROUP BY df_transaction.user_id 
     	)
    
    
    SELECT all_info.USER_ID , all_info.DEPOSITS
    FROM all_info
    WHERE 
    all_info.DEPOSITS >= 2000  AND
    
    all_info.user_id not in (
      SELECT df_transaction.user_id 
        FROM df_user_info 
        INNER JOIN df_transaction
        ON df_user_info.user_id = df_transaction.user_id
        WHERE 
        (event_kind == "crypto_deposit" AND
        strftime('%Y', onboarding_completed_at) - strftime('%Y', timestamp) == 0 AND
        strftime('%m', onboarding_completed_at) - strftime('%m', timestamp) == 0 AND
        strftime('%d', onboarding_completed_at) - strftime('%d', timestamp) <= 4) 
        GROUP BY df_transaction.user_id 
    ) 
    AND
    all_info.user_id in (
      SELECT df_transaction.user_id 
        FROM df_user_info 
        INNER JOIN df_transaction
        ON df_user_info.user_id = df_transaction.user_id
        WHERE 
        (event_kind == "fiat_deposit" AND
        strftime('%Y', onboarding_completed_at) - strftime('%Y', timestamp) == 0 AND
        strftime('%m', onboarding_completed_at) - strftime('%m', timestamp) == 0 AND
        strftime('%d', onboarding_completed_at) - strftime('%d', timestamp) <= 4) 
        GROUP BY df_transaction.user_id 
    ) 
    GROUP BY all_info.user_id 

    '''
ps.sqldf(QUERY)

,user_id,DEPOSITS
0,0055cee0fe0f41c7d782e9f7e35a92331ddeb503523f4f...,4952.391804
1,010c2c33e0538f313e641b52685f0437d5e14648790d90...,11608.562323
2,011ba7b51cb808b51202b474cb6641fa4fcb98537c07c4...,31064.072493
3,0123fae40ea35eaa408c8a3d62e2dd55a5a0814280a290...,127223.151722
4,0151e8755171fa4b8deafa8e2796a2ada805d68b93cb4f...,13848.225675
...,...,...
476,fdd23890e4772770a0e05e8d35ac61f9a01f38f752aa61...,5146.782731
477,fe6b59763e908fbf131de35ada1e94324ead0473f2e2bf...,9704.579566
478,fe79072bd100c4f2e7a70789c30a23c2c830f33beddc69...,4128.235222
479,fecc90194a2f94b1565b71573c929e550e32e02d93d5d8...,14141.838614


In [98]:
# b)
QUERY = '''
WITH 
	D1 AS (
      SELECT *, SUM(amount_in_eur) AS total_EXCHANGE_NOT_BTC_ETH 
      FROM df_transaction 
      WHERE event_kind == "exchange" AND
      SUBSTRING(currency, Instr(currency,'_' )+ 1, 4) not in ('BTC', 'ETH') 
      GROUP by user_id
    ),
    
    D2 as(
    SELECT * , SUM(amount_in_eur) AS total_EXCHANGE_BTC_ETH
      FROM df_transaction
      WHERE event_kind == "exchange" AND
      SUBSTRING(currency, Instr(currency,'_' )+ 1, 4) in ('BTC', 'ETH') 
      GROUP BY user_id
    ),

    D3 as (
    SELECT D1.user_id, total_EXCHANGE_BTC_ETH , total_EXCHANGE_NOT_BTC_ETH
      FROM D1       
      INNER JOIN D2
      ON D1.user_id = D2.user_id
    )
    
    SELECT D3.user_id ,   (total_EXCHANGE_BTC_ETH) /(total_EXCHANGE_NOT_BTC_ETH + total_EXCHANGE_BTC_ETH) * 100 AS transaction_percent_to_BTC_ETH
    FROM D3
    WHERE transaction_percent_to_BTC_ETH >= 90
    GROUP BY user_id

'''
ps.sqldf(QUERY)

,user_id,transaction_percent_to_BTC_ETH
0,08e2982d98ed9e28c8b81ad02e82eaaa6b1e18fbb4ceae...,95.255644
1,0b0397abf268fa061178916887d012511c6d7a2082882b...,93.301979
2,0c2580b7f41eeee73c584a6a98ff5be98fc5d1e47b5047...,99.146614
3,11255c97fc39a569e587b80631f68bbaca0699c6556e03...,99.402573
4,1240b4902df22528fec0760b8e3f52063f5e88a7ac0ad1...,91.590240
...,...,...
96,f8c4eb9077d362e4c92095ec9aa55ac6719065682dca1c...,90.528599
97,f8f05e9d16eac5300ae913792933d6a829d01a70272be4...,95.031265
98,fb0e55e523986cd406ef034f0ae55a0f132935dc4ce03d...,97.698761
99,fb9b94800ef67fc9ed7431a763aaf38e5028570fa92cad...,94.536526


In [246]:
# c)

QUERY = '''
WITH 
    D1 AS (
        SELECT * 
      	FROM df_transaction
       	WHERE event_kind == 'crypto_withdrawal'
        group by transaction_id
    ),
    
    D2 AS (
       SELECT * , min(timestamp) as min_timestamp FROM D1
       group BY user_id 
    ),
    
    D3 AS(
        SELECT * FROM D2
        WHERE currency IN ('BTC', 'ETH') AND
      	amount_in_eur >= 2000 
    ),
    
    D4 AS ( 
      	SELECT * 
      	FROM df_transaction
      	WHERE event_kind == 'fiat_deposit'
      	GROUP by user_id
    ),
    
    D5 as (
    SELECT * FROM D3
    left JOIN D4
    ON D3.user_id = D4.user_id
    )
    

    SELECT user_id, event_kind, amount_in_eur FROM D5
    WHERE
        (
        strftime('%Y', "timestamp:1") - strftime('%Y', timestamp) == 0 AND
        strftime('%m', "timestamp:1") - strftime('%m', timestamp) == 0 AND
        strftime('%d', "timestamp:1") - strftime('%d', timestamp) <= 4) 
    
   
    '''
ps.sqldf(QUERY)



,user_id,event_kind,amount_in_eur
0,0498a9869ae99cd6583370de65b9740feb3b10d06d622e...,crypto_withdrawal,5230.978066
1,04cdbd34ec61d770ddd471988e8af74af1cb067e35e8a3...,crypto_withdrawal,4284.327626
2,09b38ca75f3ae7f609efdba3f3f4247de721b7ef74a054...,crypto_withdrawal,2018.930606
3,12f7208772d5848f6b75ae1f94eee45f4e8d969f9ed52c...,crypto_withdrawal,3890.174093
4,1835d9d873bf1f93f67ad2e982f5961b72e9eddb30844d...,crypto_withdrawal,2583.421852
5,1e746d79a8dbd9fe368881ec7682e8d0e4b41a5f3a84cc...,crypto_withdrawal,3169.014939
6,23ce9059caeb5d58f45ae72723b54ac2e6ce3f182eec58...,crypto_withdrawal,2323.983109
7,243efd66c0cc48835b1765bb6c7569dbd5f19ac45d5757...,crypto_withdrawal,3147.917533
8,29429ef9794f788f7cc92f8cfe359b323a957df92fba7f...,crypto_withdrawal,2135.727871
9,2d0804c91bacf32f35a0188ad5db4e9ffd396b38a99a9e...,crypto_withdrawal,8440.809322
